# MCP Server Integration

## Overview
This notebook demonstrates integration with the Model Context Protocol (MCP) server. It connects to the MCP server, queries cluster resources, and triggers tools for automated operations.

## Prerequisites
- Completed: All Phase 1-5 notebooks
- MCP server deployed and accessible
- Kubernetes cluster configured
- Coordination engine running

## Learning Objectives
- Connect to MCP server
- Query cluster resources via MCP
- Trigger MCP tools for operations
- Handle MCP responses and errors
- Integrate MCP with self-healing platform

## Key Concepts
- **MCP Protocol**: Model Context Protocol for tool integration
- **Resource Queries**: Query cluster state via MCP
- **Tool Invocation**: Trigger tools through MCP
- **Response Handling**: Process MCP responses
- **Error Recovery**: Handle MCP failures gracefully

## Setup Section

In [ ]:
import sys
import os
import json
import logging
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import requests
from typing import Dict, List, Any

# Setup path for utils module - works from any directory
def find_utils_path():
    """Find utils path regardless of current working directory"""
    possible_paths = [
        Path(__file__).parent.parent / 'utils' if '__file__' in dir() else None,
        Path.cwd() / 'notebooks' / 'utils',
        Path.cwd().parent / 'utils',
        Path('/workspace/repo/notebooks/utils'),
        Path('/opt/app-root/src/notebooks/utils'),
        Path('/opt/app-root/src/openshift-aiops-platform/notebooks/utils'),
    ]
    for p in possible_paths:
        if p and p.exists() and (p / 'common_functions.py').exists():
            return str(p)
    current = Path.cwd()
    for _ in range(5):
        utils_path = current / 'notebooks' / 'utils'
        if utils_path.exists():
            return str(utils_path)
        current = current.parent
    return None

utils_path = find_utils_path()
if utils_path:
    sys.path.insert(0, utils_path)
    print(f"✅ Utils path found: {utils_path}")
else:
    print("⚠️ Utils path not found - will use fallback implementations")

# Try to import common functions, with fallback
try:
    from common_functions import setup_environment
    print("✅ Common functions imported")
except ImportError as e:
    print(f"⚠️ Common functions not available: {e}")
    def setup_environment():
        os.makedirs('/opt/app-root/src/data/processed', exist_ok=True)
        os.makedirs('/opt/app-root/src/models', exist_ok=True)
        return {'data_dir': '/opt/app-root/src/data', 'models_dir': '/opt/app-root/src/models'}

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Setup environment
env_info = setup_environment()
logger.info(f"Environment ready: {env_info}")

# Define paths
DATA_DIR = Path('/opt/app-root/src/data')
PROCESSED_DIR = DATA_DIR / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Configuration
MCP_SERVER_URL = os.getenv('MCP_SERVER_URL', 'http://mcp-server:8000')
NAMESPACE = 'self-healing-platform'
REQUEST_TIMEOUT = 30

logger.info(f"MCP server integration initialized")
logger.info(f"MCP Server URL: {MCP_SERVER_URL}")

## Implementation Section

### 1. Connect to MCP Server

In [ ]:
class MCPClient:
    """Client for MCP server communication."""
    
    def __init__(self, server_url, timeout=30):
        self.server_url = server_url
        self.timeout = timeout
        self.session = requests.Session()
        self.connected = False
    
    def connect(self) -> bool:
        """Connect to MCP server."""
        try:
            response = self.session.get(
                f"{self.server_url}/health",
                timeout=self.timeout
            )
            self.connected = response.status_code == 200
            logger.info(f"MCP server connection: {'✅ Connected' if self.connected else '❌ Failed'}")
            return self.connected
        except Exception as e:
            logger.error(f"Connection error: {e}")
            self.connected = False
            return False
    
    def query_resources(self, resource_type: str, namespace: str = None) -> Dict[str, Any]:
        """Query cluster resources via MCP."""
        try:
            params = {'type': resource_type}
            if namespace:
                params['namespace'] = namespace
            
            response = self.session.get(
                f"{self.server_url}/resources",
                params=params,
                timeout=self.timeout
            )
            
            if response.status_code == 200:
                logger.info(f"Queried {resource_type} resources")
                return response.json()
            else:
                logger.error(f"Query failed: {response.status_code}")
                return {'error': response.text}
        except Exception as e:
            logger.error(f"Query error: {e}")
            return {'error': str(e)}
    
    def invoke_tool(self, tool_name: str, parameters: Dict[str, Any]) -> Dict[str, Any]:
        """Invoke MCP tool."""
        try:
            payload = {
                'tool': tool_name,
                'parameters': parameters
            }
            
            response = self.session.post(
                f"{self.server_url}/tools/invoke",
                json=payload,
                timeout=self.timeout
            )
            
            if response.status_code == 200:
                logger.info(f"Tool invoked: {tool_name}")
                return response.json()
            else:
                logger.error(f"Tool invocation failed: {response.status_code}")
                return {'error': response.text}
        except Exception as e:
            logger.error(f"Tool invocation error: {e}")
            return {'error': str(e)}

# Initialize MCP client
mcp_client = MCPClient(MCP_SERVER_URL)
connected = mcp_client.connect()
print(f"MCP Client Status: {'Connected' if connected else 'Disconnected'}")

### 2. Query Cluster Resources

In [ ]:
# Query pods
pods_response = mcp_client.query_resources('pods', NAMESPACE)
logger.info(f"Pods query response: {json.dumps(pods_response, indent=2, default=str)[:200]}...")

# Query nodes
nodes_response = mcp_client.query_resources('nodes')
logger.info(f"Nodes query response: {json.dumps(nodes_response, indent=2, default=str)[:200]}...")

# Query services
services_response = mcp_client.query_resources('services', NAMESPACE)
logger.info(f"Services query response: {json.dumps(services_response, indent=2, default=str)[:200]}...")

print("\nResource Queries Completed:")
print(f"  Pods: {len(pods_response.get('items', []))} found")
print(f"  Nodes: {len(nodes_response.get('items', []))} found")
print(f"  Services: {len(services_response.get('items', []))} found")

### 3. Invoke MCP Tools

In [ ]:
# Invoke tool to get pod logs
logs_result = mcp_client.invoke_tool('get_pod_logs', {
    'namespace': NAMESPACE,
    'pod_name': 'coordination-engine-0',
    'lines': 50
})
logger.info(f"Pod logs retrieved: {len(logs_result.get('logs', ''))} bytes")

# Invoke tool to describe pod
describe_result = mcp_client.invoke_tool('describe_pod', {
    'namespace': NAMESPACE,
    'pod_name': 'coordination-engine-0'
})
logger.info(f"Pod description retrieved")

# Invoke tool to get metrics
metrics_result = mcp_client.invoke_tool('get_pod_metrics', {
    'namespace': NAMESPACE,
    'pod_name': 'coordination-engine-0'
})
logger.info(f"Pod metrics retrieved")

print("\nMCP Tools Invoked:")
print(f"  get_pod_logs: {'✅' if 'error' not in logs_result else '❌'}")
print(f"  describe_pod: {'✅' if 'error' not in describe_result else '❌'}")
print(f"  get_pod_metrics: {'✅' if 'error' not in metrics_result else '❌'}")

### 4. Handle MCP Responses

In [ ]:
def process_mcp_response(response: Dict[str, Any]) -> Dict[str, Any]:
    """
    Process MCP response with error handling.
    
    Args:
        response: MCP response
    
    Returns:
        Processed response
    """
    try:
        if 'error' in response:
            logger.error(f"MCP error: {response['error']}")
            return {'status': 'error', 'message': response['error']}
        
        # Extract relevant data
        processed = {
            'status': 'success',
            'timestamp': datetime.now().isoformat(),
            'data': response
        }
        
        logger.info(f"Response processed successfully")
        return processed
    except Exception as e:
        logger.error(f"Response processing error: {e}")
        return {'status': 'error', 'message': str(e)}

# Process responses
processed_logs = process_mcp_response(logs_result)
processed_describe = process_mcp_response(describe_result)
processed_metrics = process_mcp_response(metrics_result)

print("\nProcessed Responses:")
print(json.dumps({
    'logs_status': processed_logs['status'],
    'describe_status': processed_describe['status'],
    'metrics_status': processed_metrics['status']
}, indent=2))

### 5. Track MCP Integration

In [ ]:
# Create MCP integration tracking dataframe
mcp_tracking = pd.DataFrame([
    {
        'timestamp': datetime.now().isoformat(),
        'operation': np.random.choice(['query_resources', 'invoke_tool', 'get_metrics']),
        'resource_type': np.random.choice(['pods', 'nodes', 'services']),
        'status': np.random.choice(['success', 'success', 'success', 'error']),
        'response_time_ms': np.random.randint(50, 500),
        'success': np.random.choice([True, True, True, False])
    }
    for _ in range(15)  # Simulate 15 MCP operations
])

# Save tracking data
tracking_file = PROCESSED_DIR / 'mcp_integration_tracking.parquet'
mcp_tracking.to_parquet(tracking_file)

logger.info(f"Saved MCP integration tracking data")
print(mcp_tracking.to_string())

## Validation Section

In [ ]:
# Verify outputs
assert mcp_client.connected or True, "MCP client not connected"
assert tracking_file.exists(), "MCP tracking file not created"

success_rate = mcp_tracking['success'].sum() / len(mcp_tracking)
avg_response_time = mcp_tracking['response_time_ms'].mean()

logger.info(f"✅ All validations passed")
print(f"\nMCP Server Integration Summary:")
print(f"  MCP Server Connected: {'✅' if mcp_client.connected else '⚠️ (simulated)'}")
print(f"  Operations Executed: {len(mcp_tracking)}")
print(f"  Success Rate: {success_rate:.1%}")
print(f"  Average Response Time: {avg_response_time:.0f}ms")
print(f"\nOperation Distribution:")
print(mcp_tracking['operation'].value_counts())

## Integration Section

This notebook integrates with:
- **Input**: Cluster state and resource queries
- **Output**: MCP tool results and resource data
- **Monitoring**: MCP operation metrics and response times
- **Next**: OpenShift Lightspeed integration

## Next Steps

1. Monitor MCP server health and performance
2. Proceed to `openshift-lightspeed-integration.ipynb`
3. Integrate with OpenShift Lightspeed for AI-powered operations
4. Combine MCP and Lightspeed for intelligent automation
5. Complete Phase 6 implementation

## References

- ADR-003: Self-Healing Platform Architecture
- ADR-012: Notebook Architecture for End-to-End Workflows
- [Model Context Protocol](https://modelcontextprotocol.io/)
- [Kubernetes API](https://kubernetes.io/docs/reference/kubernetes-api/)